In [1]:
import cv2 as cv
import mediapipe as mp
import torch
from models.hand_gesture import HandGesture
from models.hand_gesture import normalized_landmarks, landmark_bbox

In [2]:
PATH = 'data/videos/test.mp4'
PATH_GESTURE_MODEL = 'models/hand_gesture_model.pth'

In [3]:
video = cv.VideoCapture(PATH)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

Trail = []
my_gesture = HandGesture(PATH_GESTURE_MODEL)
while video.isOpened():
    ret, frame = video.read()
    if ret:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        H, W, _ = frame.shape
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                position = normalized_landmarks(hand_landmarks, H, W)
                pred, confidence = my_gesture.predict(position)
                pred = pred.replace('_', ' ')
                if pred == 'index finger up' and confidence > 0.98:
                    index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    cx, cy = int(index_finger.x * W), int(index_finger.y * H)
                    Trail.append((cx, cy))

                x1, y1, x2, y2 = landmark_bbox(hand_landmarks, H, W)
                cv.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 0), 4)
                cv.putText(frame, f'{pred} | confidence : {confidence:0.3}', (int(x1), int(y1)),
                           cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv.LINE_AA)

                # mp_drawing.draw_landmarks(
                # frame,
                # hand_landmarks,
                # mp_hands.HAND_CONNECTIONS,  
                # mp_drawing_styles.get_default_hand_landmarks_style(),
                # mp_drawing_styles.get_default_hand_connections_style())

        for i in Trail:
            cv.circle(frame, i, 10, (0, 255, 0), cv.FILLED)
        cv.imshow('frame', frame)

        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
video.release()
cv.destroyAllWindows()